In [1]:
import numpy as np
import pandas as pd


#ProPublica data from here https://projects.propublica.org/colleges
#Not used in this analysis - but maybe would be better?

pp=pd.read_csv("../../data/raw/compas/ProPublica_Data.csv")
pp.head()

,State,Percentage of Pell Grantees,Median Federal Debt for All Students,Median Federal Debt of Pell Grantees,Nonrepayment Rate for Pell Students,Nonrepayment Rate for All Students,"Average Cost for Low-Income Students, Per Year",Default Rate for All Students,"Percentage of All Students Earning $25K or Less Per Year, 10 Years After Enrolling"
0,Adelphi University,0.308,25000.0,23000.0,0.363,0.297,22197.0,0.032,0.204
1,Alliant International University-San Diego,0.123,3000.0,13425.0,0.634,0.496,16785.0,0.027,0.197
2,American International College,0.506,27000.0,21750.0,0.586,0.519,17160.0,0.062,0.263
3,American University,0.167,24589.0,25000.0,0.273,0.199,28524.0,0.021,0.168
4,Andrews University,0.293,27750.0,26768.0,0.536,0.493,19226.0,0.053,0.338


In [2]:
usnews = pd.read_csv("../../data/raw/college/USNewsCollegeRankings2018.csv")
usnews.set_index('University', inplace=True)
usnews.head()

,City,State,Rank,In-state Tuition,Out-state Tuition,Enrollment
University,,,,,,
Princeton University,Princeton,NJ,1,$47140,$47140,5400.0
Harvard University,Cambridge,MA,2,$48949,$48949,6710.0
University of Chicago,Chicago,IL,3,$54825,$54825,5941.0
Yale University,New Haven,CT,3,$51400,$51400,5472.0
Columbia University,New York,NY,5,$57208,$57208,6113.0


In [3]:
scorecard = pd.read_csv("../../data/raw/college/MERGED2015_16_PP.csv")
scorecard.shape

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,9,1725,1726,1727,1728,1729,1743) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(7593, 1777)

In [4]:
usnews=usnews[usnews['Rank']!= 'Rank Not Published']
usnews=usnews[usnews['Rank']!= 'Unranked']
usnews.shape

(230, 6)

In [5]:
drop=['UNITID','OPEID','OPEID6','CITY','STABBR','ZIP','ACCREDAGENCY','INSTURL','NPCURL']
scorecard.drop(drop, axis=1, inplace=True)
scorecard.set_index('INSTNM',inplace=True)
scorecard = scorecard.loc[usnews.index]
scorecard['Rank']=usnews['Rank']
scorecard.shape

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  after removing the cwd from sys.path.


(230, 1768)

In [6]:
# scorecard.dropna(axis=1,thresh=100).shape
data = scorecard.dropna(thresh=100, axis=1).dropna(thresh=500)
for c in data.columns.values:
    data[c] = pd.to_numeric(data[c], errors='coerce')
data.dropna(axis=1,thresh=100, inplace=True)
# data.fillna(data.mean, inplace=True)
cols=[]
for c in data.columns.values:
    if data[c].var()!=0:
       cols.append(c)
data=data[cols]
data.shape

(142, 441)

In [7]:
def normalize(df):
    return(df - df.mean()) / df.std()

scnorm = normalize(data)

Feature Analysis

In [10]:
from scipy import stats
#Find features that are correlated
names = pd.read_csv('../../data/raw/college/col_names.csv')

#Remember that rankings are inverted - #1 best

pos_corr_features = []
neg_corr_features = []

for c in data.columns.values:
    corr, p = stats.pearsonr(data[c], data['Rank'])
    if p <0.05 and corr > 0.6 :
        pos_corr_features.append([names[names['VARIABLE NAME']==c]['developer-friendly name'].max(),c, corr,p])

for c in data.columns.values:
    corr, p = stats.pearsonr(data[c], data['Rank'])
    if p <0.05 and corr <-0.6 :
        neg_corr_features.append([names[names['VARIABLE NAME']==c]['developer-friendly name'].max(),c, corr,p])

print("Negatively correlated features")
print
for i in neg_corr_features:
    print(i)
print
print("Positively correlated features")
print
for i in pos_corr_features:
    print(i)

Negatively correlated features
['attendance.academic_year', 'COSTT4_A', -0.6932427662378435, 1.1747684869920824e-21]
['tuition.in_state', 'TUITIONFEE_IN', -0.6708655538227406, 6.604543928798278e-20]
['tuition.out_of_state', 'TUITIONFEE_OUT', -0.8038813264738107, 2.1484910350060433e-33]
['tuition_revenue_per_fte', 'TUITFTE', -0.6832112699637477, 7.476580762716287e-21]
['instructional_expenditure_per_fte', 'INEXPFTE', -0.6520470892717466, 1.5058505552534478e-18]
['faculty_salary', 'AVGFACSAL', -0.7994718046942055, 8.630216105012225e-33]
['completion_rate_4yr_150nt', 'C150_4', -0.9391448493466977, 7.735549844824654e-67]
['completion_rate_4yr_150nt_pooled', 'C150_4_POOLED', -0.9402921127190229, 2.1249830406631032e-67]
['share_first.time_full.time', 'PFTFTUG1_EF', -0.7212443560926453, 4.3840422394709534e-24]
['completion_rate_4yr_150_black', 'C150_4_BLACK', -0.8228389446670504, 3.544033330505613e-36]
['completion_rate_4yr_150_hispanic', 'C150_4_HISP', -0.8482352985059171, 1.7920993301776731

In [17]:
scnorm.sort_index(inplace=True)
# create groups based on median family income - high, low
groups = pd.DataFrame()
groups['PCTPELL']= data['PCTPELL']
data.drop(['PCTPELL'], axis=1, inplace=True)
m = groups['PCTPELL'].mean()
groups.loc[groups['PCTPELL'] >m,'Group'] = 1 #many grants
groups.loc[groups['PCTPELL'] <=m,'Group'] = 0 #few grants

scnorm_groups = pd.DataFrame(scnorm)
scnorm_groups['Group'] = groups['Group']
X = np.nan_to_num(scnorm_groups.drop(['Rank'], axis=1))
y = np.nan_to_num(scnorm['Rank'])
g=groups['Group']
# groups.head()

KeyError: 'PCTPELL'

In [16]:
X_all = pd.DataFrame(X)
X['Group'] = g

print (X)
# print (X_all)

[[-0.26726291  0.79229534  0.48719052 ... -0.73179423 -0.61734258
  -0.67537433]
 [-0.26726291  0.79229534 -1.18384943 ... -0.45430117  0.25130789
  -0.01195442]
 [-0.26726291  0.79229534 -0.18122546 ... -1.06299563 -0.34150796
  -0.85266957]
 ...
 [-0.26726291  0.79229534 -0.24806706 ... -0.81983377 -0.49085488
  -0.70526847]
 [-0.26726291  0.79229534 -1.31753263 ... -0.68903818 -0.67068076
   0.34769506]
 [-0.26726291  0.79229534  0.48719052 ... -0.97242863  0.
  -0.43392168]]


In [ ]:
import pickle



pickle.dump( X, open( "x.p", "wb" ) )
pickle.dump( y, open( "y.p", "wb" ) )
pickle.dump( g, open( "g.p", "wb" ) )